# Query Service
Query automatic synaptic partner predictions and integrate with CATMAID

October 2019; Stephan Gerhard, Julia Buhmann, Philipp Schlegel, Jan Funke

## Queries

  1. get all connections between a given pair of skeletons,
  2. get all upstream connections for a given skeleton
  3. get all downstream connections for a given skeleton
  4. get all upstream/downstream connections for a given skeleton

## Imports

In [4]:
# pip install git+git://github.com/schlegelp/pymaid@master
import pymaid
import requests
import fafbseg
import json
import numpy as np
import pandas as pd

In [8]:
conf = json.load(open('/home/stephan/.catmaidapi.json','r'))

## Configuration

In [41]:
# Source CATMAID instance and project to retrieve skeletons from
source_catmaid_url = conf['url']
source_catmaid_project_id = conf['project_id']
source_catmaid_http_user = None
source_catmaid_http_password = None
source_catmaid_access_token = conf['access_token']

# Set of neurons to get from the source project by skeleton ids
source_skeleton_ids = [733084786, 733084781]

# Target CATMAID instance and project to push skeletons with synapses to
target_catmaid_url = ''
target_catmaid_project_id = ''
target_catmaid_http_user = ''
target_catmaid_http_password = ''
target_catmaid_access_token = ''

# CloudVolume Server URL
segmentation_server_url = "https://storage.googleapis.com/fafb-ffn1-20190805/segmentation"

In [42]:
source_catmaid = pymaid.CatmaidInstance(
                            source_catmaid_url,
                            source_catmaid_http_user,
                            source_catmaid_http_password,
                            source_catmaid_access_token)

# only one CATMAID instance
target_catmaid = source_catmaid

"""
target_catmaid = pymaid.CatmaidInstance(
                            target_catmaid_url,
                            target_catmaid_http_user,
                            target_catmaid_http_password,
                            target_catmaid_access_token)
"""

INFO  : Global CATMAID instance set. Caching is ON. (pymaid)


'\ntarget_catmaid = pymaid.CatmaidInstance(\n                            target_catmaid_url,\n                            target_catmaid_http_user,\n                            target_catmaid_http_password,\n                            target_catmaid_access_token)\n'

In [43]:
fafbseg.use_google_storage(segmentation_server_url)

Using Google CloudStorage to retrieve segmentation IDs.


### Get skeletons from CATMAID

In [44]:
neuron_list = pymaid.get_neuron(source_skeleton_ids)

### Look up segmentation IDs for skeletons

In [51]:
n.nodes[['x','y','z']].values

array([[475416, 244440, 110120],
       [474952, 244920, 110120],
       [474512, 245176, 110120]])

In [45]:
for n in neuron_list:
    print(n)
    # Add segmentation IDs as new column to node table (pandas DataFrame)
    n.nodes['seg_id'] = fafbseg.segmentation.get_seg_ids(n.nodes[['x','y','z']].values)

Segmentation IDs:   0%|          | 0/2 [00:00<?, ?it/s]

type              <class 'pymaid.core.CatmaidNeuron'>
neuron_name                          neuron 733084787
skeleton_id                                 733084786
n_nodes                                             4
n_connectors                                        0
n_branch_nodes                                      0
n_end_nodes                                         1
n_open_ends                                         1
cable_length                                  1.04626
review_status                                      NA
soma                                             None
dtype: object


Segmentation IDs: 100%|██████████| 2/2 [00:00<00:00, 77.12it/s]

type              <class 'pymaid.core.CatmaidNeuron'>
neuron_name                          neuron 733084782
skeleton_id                                 733084781
n_nodes                                             3
n_connectors                                        0
n_branch_nodes                                      0
n_end_nodes                                         1
n_open_ends                                         1
cable_length                                  1.17666
review_status                                      NA
soma                                             None
dtype: object


In [46]:
neuron_list

,neuron_name,skeleton_id,n_nodes,n_connectors,n_branch_nodes,n_end_nodes,open_ends,cable_length,review_status,soma
0,neuron 733084787,733084786,4,0,0,1,1,1.046257,NA,False
1,neuron 733084782,733084781,3,0,0,1,1,1.176659,NA,False


In [47]:
neuron_list["neuron 733084787"].nodes

,confidence,creator_id,parent_id,radius,seg_id,skeleton_id,treenode_id,type,x,y,z
0,5,2,1963074702,-1,5018650837,733084786,1963074703,end,473916,244404,110120
1,5,2,1963074701,-1,5018650837,733084786,1963074702,slab,473676,244196,110120
2,5,2,1963074700,-1,5018650837,733084786,1963074701,slab,473220,244044,110120
3,5,2,None,-1,5018650837,733084786,1963074700,root,472972,244044,110120


In [48]:
neuron_list["neuron 733084782"].nodes

,confidence,creator_id,parent_id,radius,seg_id,skeleton_id,treenode_id,type,x,y,z
0,5,2,1963074698,-1,3708389819,733084781,1963074699,end,475416,244440,110120
1,5,2,1963074697,-1,3708389819,733084781,1963074698,slab,474952,244920,110120
2,5,2,None,-1,0,733084781,1963074697,root,474512,245176,110120


### Retrieve all synaptic links for the set of segmentationIDs

@juliab: Can one get all the relevant synaptic links for a set of skeletons on-the-fly, and only do the segmentID lookup for those, or is everything needed? If everything is needed, do you have the output segmentation ID for all pre/post locations for all links?

Alternative: precomputed, e.g. from a local sqlite database with the prepopulated lookups from synaptic link to segmentation IDs

In [74]:
allsynapses = pd.read_csv('synlinks_testdata.csv', index_col='index')

In [75]:
allsynapses

,ids,pre_x,pre_y,pre_z,post_x,post_y,post_z,scores
index,,,,,,,,
0,612550824501693,91295,61163,1847,91316,61158,1848,7.635252
1,610576269929504,90970,61389,1848,90932,61376,1848,15.259161
2,608119945985227,90887,61205,1848,90912,61188,1849,10.514650
3,605977555068920,90858,61100,1854,90845,61069,1854,71.585907
4,608262160558845,90954,61169,1855,90926,61181,1854,21.044254
5,610148599914822,91054,61198,1858,91071,61192,1857,9.547329
6,617924996725302,91373,61515,1858,91379,61536,1857,389.255432
7,617110893498108,91354,61493,1859,91340,61506,1858,699.752563
8,618032800075034,91372,61522,1859,91372,61551,1858,14.338421


In [87]:
# select synapses that are close to the set of skeletons
filtered_synapses = allsynapses[:3]

In [92]:
# get segmentation ID for all pre and post locations
filtered_synapses.loc[:,'pre_sv_id'] = \
    fafbseg.segmentation.get_seg_ids( \
        np.multiply(filtered_synapses[['pre_x','pre_y','pre_z']].values, np.array([4,4,40])))

Segmentation IDs: 100%|██████████| 2/2 [00:03<00:00,  1.96s/it]


In [93]:
filtered_synapses.loc[:,'post_sv_id'] = \
    fafbseg.segmentation.get_seg_ids( \
    np.multiply(filtered_synapses[['post_x','post_y','post_z']].values, np.array([4,4,40])))

Segmentation IDs: 100%|██████████| 2/2 [00:00<00:00, 57.18it/s]


In [94]:
filtered_synapses

,ids,pre_x,pre_y,pre_z,post_x,post_y,post_z,scores,pre_sv_id,post_sv_id
index,,,,,,,,,,
0,612550824501693,91295,61163,1847,91316,61158,1848,7.635252,3320006144,3062184700
1,610576269929504,90970,61389,1848,90932,61376,1848,15.259161,3452426085,3452426084
2,608119945985227,90887,61205,1848,90912,61188,1849,10.514650,3452426084,3452426130


### Mapping the synaptic links with the corresponding skeleton

At this point, we'd have the set of skeleton (skeleton nodes) and synaptic links (pre/post locations) to do the matching based on matching IDs

Discard synaptic links that stem from split-away parts of segments with false mergers

In [ ]:
# TO EXTRACT from exiting code

### Filter synaptic links

Possible filtering options:
* Filter same ID (if pre and postsynaptic site land in the same segment / are mapped onto the same skeleton)
* Filter synapses that have the same underlying connectivity and that are in close distance (duplicated detection)

### Push the skeleton data with synapses to CATMAID using PyMAID

Pushing connections to CATMAID is a two-step process: creating connectors and linking them.
pymaid.add_connector takes an array x/y/z coordinates and returns the IDs of the newly created connectors
pymaid.link_connector takes a list of [(treenode_id, connector_id, RELATION), ...](with RELATION being 'presynaptic_to' or 'postsynaptic_to')
So all we need is a map of presynaptic_treenode_ID -> connector_position/_ID -> postsynaptic_treenode_ID